In [1]:
import org.apache.spark._
import scala.io.Source

## Task
The task is to implement the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms. 

![bigpicture](./docs/bigpicture.png/)

## Data
The sample data are textfiles of hotel reviews. They are loaded into a dataframe, textfiles.

In [15]:
// val documents = sc.textFile("../data/OpinRankDataset/hotels/small_sample/*").toDF
val documents = Source.fromFile("../data/OpinRankDataset/hotels/small_sample/usa_san francisco_abigail_hotel").getLines()

documents.take(1).foreach(println)

Oct 30 2009 	OK value though shabby; good position.	I extended a one night stay to three. The staff were basically helpful when asked. My room was clean, comfortable, and daily serviced. I think the position is good for a budget hotel, near the Golden Gate theatre, right by Asian Arts Museum and Civic Centre. It's an old and architecturally atmospheric hotel.Negatives: it is very badly neglected. Things that might put you off are the need usually to ring the outside doorbell and wait to get someone to the unattended reception, unreliable breakfast availability (nil when I was there but I can see from other reviews that it can appear!), a very unreliable (but rather beautiful) classic Otis elevator, some very dilapidated corridors on the way to your room when the elevator is kaput, shabby though formerly elegant decor, homeless on nearby streets, though the street immediately outside was clear and felt safe and I didn't see any drug taking or pushing and I was never bothered anywhere. H

documents = non-empty iterator


non-empty iterator

In [27]:
import scala.util.hashing
import scala.math

class Shinglesss extends Serializable {
    def docToShingles(doc: String, k: Integer) : (Set[Integer]) = {

        var shingles : Set[String] = Set()

        // number of shingles = length/k
        val n = doc.length/k

        // for loop slice string
        for( i <- 0 to (n-1)){
           shingles += doc.slice(k * i, k * (i+1))
        }
        // hash each shingle in the set
        val hashedShingles = shingles.map(x => hash(x))
        hashedShingles
    }
    def hash(s: String) : (Integer) = {
      // Hashes a string to one out of 2^32 buckets with mod(2^32)
      val n = Math.pow(2, 32)
      val intRep = s.getBytes.foldLeft(0)(_+_).asInstanceOf[Integer]
      val hash = (intRep % n).asInstanceOf[Integer]
      hash
    }

    def compareSets(s1: Array[Integer], s2: Array[Integer]) : (Double) = {
        // comparesets that compares two sets of shingles and returns the similarity
        
        val union = s1.union(s2)

        val intersection = s1.intersect(s2)

        val similarity = intersection.size/union.size

        (similarity)
    }
}




// def Shingling - returns a k-shingle representation of a text document
// can either use set or multiset (allowing multi occuring shingles)







defined class Shinglesss


lastException: Throwable = null


In [32]:
val s = new Shinglesss()
val k = 10
val shingleDocs = documents.map(x => s.docToShingles(x, k))
shingleDocs.take(3).foreach(println)
// val similarities = shingleDocs.map(x => s.compareSets(x, x)).toDF

Name: java.lang.ClassCastException
Message: java.lang.Double cannot be cast to java.lang.Integer
StackTrace:   at Shinglesss.hash(<console>:40)
  at Shinglesss$$anonfun$1.apply(<console>:33)
  at Shinglesss$$anonfun$1.apply(<console>:33)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.immutable.HashSet$HashSet1.foreach(HashSet.scala:316)
  at scala.collection.immutable.HashSet$HashTrieSet.foreach(HashSet.scala:972)
  at scala.collection.immutable.HashSet$HashTrieSet.foreach(HashSet.scala:972)
  at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
  at scala.collection.AbstractSet.scala$collection$SetLike$$super$map(Set.scala:47)
  at scala.collection.SetLike$class.map(SetLike.scala:92)
  at scala.collection.AbstractSet.map(Set.scala:47)
  at Shinglesss.docToShingles(<console>:33)
  at $anonfun$1.apply(<console>:48)
  at $a

In [72]:
class NewTokenlizer10 extends Serializable{
      val tokens: (String, Integer) => Set[Integer] = (doc:String, k:Integer) => {

            var shingles : Set[String] = Set()

            // number of shingles = length/k
            val n = doc.length/k

            // for loop slice string
            for( i <- 0 to (n-1)){
               shingles += doc.slice(k * i, k * (i+1))
            }
        // hash each shingle in the set
        val hashedShingles = shingles.map(x => hash(x))
        hashedShingles
      }
    val hash: String => Integer = (s: String) => {
      // Hashes a string to one out of 2^32 buckets with mod(2^32)
      val n = Math.pow(2, 32)
      val intRep = s.getBytes.foldLeft(0)(_+_).asInstanceOf[Integer]
      val hash = (intRep % n).asInstanceOf[Integer]
      hash
    }
    val compareSets: (Set[Integer], Set[Integer]) => Double = (s1: Set[Integer], s2: Set[Integer]) => {
        // comparesets that compares two sets of shingles and returns the similarity
        
        val union = s1.union(s2)

        val intersection = s1.intersect(s2)

        val similarity = intersection.size/union.size
        println(similarity)
        (similarity)
    }
}

defined class NewTokenlizer10


In [73]:
val s = new NewTokenlizer10()
val k = 10
val shingleDocs = documents.map(x => s.tokens(x.getString(0), k))
val similarities = shingleDocs.map(x => s.compareSets(x, x)).toDF


s = NewTokenlizer10@835cc0a
k = 10
shingleDocs = [value: array<int>]
similarities = [value: double]


[value: double]

In [81]:
similarities.collect()

Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:345)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:335)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:159)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2299)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:844)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:843)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
  at org.apache.spark.rdd.RDD.mapPartitionsWithIndex(RDD.scala:843)
  at org.apache.spark.sql.execution.WholeStageCodegenExec.doExecute(WholeStageCodegenExec.scala:608)
  at org.apache

In [5]:
// def MinHashing - returns the minhash representation of a text document based on a shingling

// def CompareSignatures a method compare 

Name: Syntax Error.
Message: 
StackTrace: 

In [48]:
// Main procedure
val s = new Shingles()

val k = 10

val shingleDocs = documents.map(x => s.docToShingles(x.getString(0), k))

// val similarities = shingleDocs.map(x => s.compareSets(x, x))


shingleDocs.collect().foreach(println)

Name: Unknown Error
Message: <console>:52: error: Unable to find encoder for type stored in a Dataset.  Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.
       val shingleDocs = documents.map(x => s.docToShingles(x.getString(0), k))
                                      ^

StackTrace: 

In [ ]:
import scala.util.hashing
import scala.math

class Shinglesss extends Serializable {
    def docToShingles(doc: String, k: Integer) : (Set[Integer]) = {

        var shingles : Set[String] = Set()

        // number of shingles = length/k
        val n = doc.length/k

        // for loop slice string
        for( i <- 0 to (n-1)){
           shingles += doc.slice(k * i, k * (i+1))
        }
        // hash each shingle in the set
        val hashedShingles = shingles.map(x => hash(x))
        hashedShingles
    }
    def hash(s: String) : (Integer) = {
      // Hashes a string to one out of 2^32 buckets with mod(2^32)
      val n = Math.pow(2, 32)
      val intRep = s.getBytes.foldLeft(0)(_+_).asInstanceOf[Integer]
      val hash = (intRep % n).asInstanceOf[Integer]
      hash
    }

    def compareSets(s1: Array[Integer], s2: Array[Integer]) : (Double) = {
        // comparesets that compares two sets of shingles and returns the similarity
        
        val union = s1.union(s2)

        val intersection = s1.intersect(s2)

        val similarity = intersection.size/union.size

        (similarity)
    }
}




// def Shingling - returns a k-shingle representation of a text document
// can either use set or multiset (allowing multi occuring shingles)





